## Acceptability, feasibility, and user experiences of 💎 NOLA Gem

_WIP - PRESENTLY AT R&R_

_Viz. and local LLM–assisted deductive qual coding scripts for pilot trial (N = 32) analyses of acceptability, feasibility, and user experience with [💎 NOLA Gem](https://www.researchprotocols.org/2023/1/e47151/authors): a geospatially customizable culturally tailored JITAI for violence-affected people living with HIV._

> `nola_gem_acceptability.ipynb`<br>
> Simone J. Skeen x Claude Code (01-28-2025)

### 1. Prepare
Installs, imports, requisite packages; customizes outputs.
***

In [ ]:
%%capture

%pip install ollama
#%pip install python-docx

In [ ]:
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import pandas as pd
import warnings

from matplotlib import cm
from matplotlib.lines import Line2D
from pathlib import Path
from sklearn.metrics import cohen_kappa_score

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.options.mode.copy_on_write = True
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

for c in (FutureWarning, UserWarning):
    warnings.simplefilter(action = 'ignore', category = c)

In [ ]:
# path configuration

### update ROOT to match your local directory structure

ROOT = Path('.')  # project root

DATA_DIR = ROOT / 'data'
DATA_PARADATA = DATA_DIR / 'paradata'
DATA_QUAL = DATA_DIR / 'qual'
FIGURES_DIR = ROOT / 'outputs' / 'figures'

### optional: custom font (uncomment and update path if needed)

#fm.fontManager.addfont('/path/to/Arial.ttf')
#plt.rcParams['font.family'] = 'Arial'

### 2. Write
Defines `qualitative.py` module.
***

In [ ]:
%%writefile qualitative.py

import requests
import json
import pandas as pd
import time
from sklearn.metrics import cohen_kappa_score

def build_prompt(code, alias, code_def, code_ex):
    '''
    Constructs a deductive coding prompt for LLM-based qualitative analysis.
    
    Parameters:
    -----------
    code : str
        Full name of the qualitative code (e.g., 'JITAI recognition').
    alias : str
        Short alias for column naming (e.g., 'strs').
    code_def : str
        Definition of what the code captures, including inclusion/exclusion criteria.
    code_ex : str
        Human-validated examples of text that should receive this code.
    
    Returns:
    --------
    str
        Complete prompt string ready for LLM consumption.
    '''
    
    role = '''
    You are tasked with applying pre-defined qualitative codes to segments of text excerpted 
    from interviews with users of a mental health app for people living with HIV. 
    
    The app included text-messaged surveys multiple times a day, as well as self-directed sessions 
    on mindfulness meditation, trauma psychoeducation (etc.), and momentary prompts to engage with 
    brief coping skills training as needed throughout the day. 
    
    You will be provided a definition, instructions, and 
    key exemplars of text to guide your coding decisions.
    '''

    text = '''
    ---
    Text to classify:
    "{text}"
    ---
    '''
    
    definition = f'''
    Definition of "{code}": {code_def}.
    '''
    
    instruction = f'''
    You will be provided with a piece of text. For each piece of text:
    - If it meets the definition of "{code}," output {alias}_llm as "1".
    - Otherwise, output {alias}_llm as "0".
    - Also provide a short explanation in exactly two sentences, stored in {alias}_expl.

    Please respond in valid JSON with keys "{alias}_llm" and "{alias}_expl" only.
    '''
    
    example = f'''
    Below are human-validated examples of "{code}"

    - "{code_ex}"
    '''
    
    return f'{role}{definition}{instruction}{text}{example}'

# ------------------------------------------------------------------------------------------------- #

def code_texts_deductively_ollama(df, alias, text_column, endpoint_url, prompt_template, model_name):
    '''
    Classifies each row of 'text' column in provided df in accord with human-specified prompt,
    includes chain-of-thought reasoning, returning explanations for classification decision.

    Parameters:
    -----------
    df : pd.DataFrame
        df containing the text to classify.
    alias : str
        alias (for brevity) of the qualitative code to be applied.
    text_column : str
        column name in df containing the text to be analyzed.
    endpoint_url : str
        URL where locally hosted LLM runs.
    prompt_template : str
        prompt text with a placeholder (e.g. '{text}') where the row's text will be inserted.
    model_name : str
        model tasked with qualitative deductive coding.

    Returns:
    --------
   df : pd.DataFrame
        The original df with two new columns per deductive code: '{alias}_llm' (either "0" or "1")
        and '{alias}_expl' (the chain-of-thought explanation)
    '''

    label_column = f'{alias}_llm'
    explanation_column = f'{alias}_expl'

    # insert cols - necessary for .update() downstream
    
    df[label_column] = None
    df[explanation_column] = None    
    
    results = []

    for idx, row in df.iterrows():
        row_text = row[text_column]

        unique_id = f"[Row ID: {idx}]"
        prompt = f"{unique_id}\n\n" + prompt_template.format(text=row_text)

        response = requests.post(
            endpoint_url,
            headers={'Content-Type': 'application/json'},
            json={
                'model': model_name,
                'prompt': prompt,
                'stream': False
            })

        print(f"\n--- Index {idx} ---")
        print("Prompt:")
        print(prompt)
        print(f"Status: {response.status_code}")
        print("Raw response:")
        print(response.text)

        label = None
        explanation = None

        if response.status_code == 200:
            try:
                result_json = response.json()
                raw_response_str = result_json.get('response', ' ')

                cleaned_str = raw_response_str.strip().replace("```json", " ").replace("```", " ").strip()
                parsed_output = json.loads(cleaned_str)

                label = parsed_output.get(label_column)
                explanation = parsed_output.get(explanation_column)

            except (json.JSONDecodeError, KeyError, TypeError) as e:
                print("JSON error:", e)
                print("Bad string:")
                print(cleaned_str)

        # save results by row index for correct matching
        
        results.append({
            'idx': idx,
            label_column: label,
            explanation_column: explanation
            })

        # impose delay - avoid model caching errors
        
        time.sleep(0.25)        
        
    # create result df - align to input df by row index
    
    result_df = pd.DataFrame(results).set_index('idx')
    df.update(result_df)

    return df

# ------------------------------------------------------------------------------------------------- #

def calculate_kappa(df, col1, col2):
    '''
    Computes Cohen's kappa between two columns.
    '''
    return cohen_kappa_score(df[col1], df[col2])

# ------------------------------------------------------------------------------------------------- #

def calculate_percent_agreement(df, col_pairs):
    '''
    Computes percent agreement for a list of column pairs.
    '''
    results = {}
    for col1, col2 in col_pairs:
        agreement = df[col1] == df[col2]
        percent_agreement = (agreement.sum() / len(df)) * 100
        results[f"{col1} & {col2}"] = percent_agreement
    return results

# ------------------------------------------------------------------------------------------------- #

def encode_disagreements(row):
    '''
    Returns 1 if two values disagree, 0 otherwise.
    '''
    return 1 if row[0] != row[1] else 0

#### Import

In [ ]:
from qualitative import( # type: ignore
    build_prompt,
    code_texts_deductively_ollama,
    calculate_kappa,
    calculate_percent_agreement,
    encode_disagreements,
    )

In [ ]:
import seaborn as sns

def plot_acceptability_scatter(df, int_cols, x_labels, y_labels, title, output_path, 
                                replace_val = None, color = '#7eb0d5', figsize = (8, 5)):
    '''
    Creates a jittered scatter plot with mean +/- SD overlay for Likert-scale acceptability data.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame containing the intervention columns.
    int_cols : list
        Column names to plot (e.g., ['int1_rev', 'int2_rev', ...]).
    x_labels : list
        Labels for x-axis ticks (must match length of int_cols).
    y_labels : dict
        Mapping of numeric values to y-axis labels (e.g., {1: 'disagree', 2: 'somewhat disagree'}).
    title : str
        Plot title.
    output_path : Path
        Full path for saving the figure.
    replace_val : int, optional
        Value to replace with NaN before plotting (e.g., 5 for "N/A" responses).
    color : str
        Hex color for points and error bars. Default: '#7eb0d5' (ng_blue).
    figsize : tuple
        Figure dimensions. Default: (8, 5).
    
    Returns:
    --------
    None (displays and saves figure)
    '''
    
    sns.set_style(style = 'whitegrid', rc = None)
    
    fig, ax = plt.subplots(figsize = figsize)
    x_positions = range(len(int_cols))
    
    for i, col in enumerate(int_cols):
        y_vals = df[col].astype(float)
        
        # handle N/A responses if specified
        if replace_val is not None:
            y_vals = y_vals.replace(replace_val, np.nan).dropna()
        
        x_jittered = np.random.normal(
            loc = i, 
            scale = 0.16, 
            size = len(y_vals),
            )
        
        # scatter points
        ax.scatter(
            x_jittered, 
            y_vals, 
            alpha = 0.6, 
            s = 35, 
            color = color, 
            linewidths = 0.5,
            )
        
        # mean +/- SD overlay
        ax.errorbar(
            i, 
            y_vals.mean(), 
            yerr = y_vals.std(), 
            fmt = 'D', 
            color = color,
            markersize = 8, 
            capsize = 0, 
            linewidth = 1, 
            zorder = 3,
            )
    
    # format axes
    ax.set_xticks(x_positions)
    ax.set_xticklabels(
        x_labels, 
        rotation = 45, 
        ha = 'right', 
        fontsize = 9,
        )
    ax.tick_params(
        axis = 'y', 
        left = False, 
        labelleft = False,
        )
    
    ax_right = ax.secondary_yaxis('right')
    ax_right.set_yticks(list(y_labels.keys()))
    ax_right.set_yticklabels(list(y_labels.values()))
    
    ax.set_title(title, fontsize = 10)
    ax.grid(False)
    plt.tight_layout()
    
    # save and display
    plt.savefig(output_path, dpi = 300)
    plt.show()

### 2. Describe
Aggregates, tabulates, paradata.
***

#### 2a. Daily diary skills recommendations

In [ ]:
d_diary = pd.read_csv(DATA_PARADATA / 'diary_response.csv')

valid_tx_ids = {
    'g1008', 'g1011', 'g1014', 'g1019', 'g1023', 'g1025', 'g1027',
    'g1030', 'g1033', 'g1034', 'g1035', 'g1037', 'g1038', 'g1039',
    'g1040', 'g1042', 'g1044', 'g1046', 'g1047', 'g1049', 'g1050',
    'g1051', 'gg1045'
    }

# reset idx

d = d_diary[d_diary['username'].isin(valid_tx_ids)]
d.reset_index(inplace = True, drop = True)

# convert datetime

d['created_at'] = pd.to_datetime(
    d['created_at'], utc = True
    ).dt.tz_convert(None).dt.strftime('%Y-%m-%d %H:%M:%S')

# inspect

d.shape
d.info()
d.head(4)

In [ ]:
#print(d.columns.tolist())
#d.dtypes

In [ ]:
# recommendation_skill_selection - inspect

d['recommendation_skill_selection'] = d['recommendation_skill_selection'].fillna('none')
print(d['recommendation_skill_selection'].unique())

In [ ]:
# get n 

n = d['recommendation_skill_selection'].value_counts(dropna = False)
#pct = d['recommendation_skill_selection'].value_counts(
#    normalize = True, 
#    dropna = False,
#    ) * 100

# merge, display

summary = pd.DataFrame({
    'n': n,
#    'pct': pct.round(2) ### round to 2 decimal places
    })
print(summary)

In [ ]:
# get %: n / N skills recommendations 

10 / 144

#### 2b. Completed skills

In [ ]:
d = pd.read_csv(DATA_PARADATA / 'UserSummary.csv')

# inspect

d.shape
d.info()
d.head(23)

In [ ]:
# * spent_minutes - mdn / iqr

skills_min = [
    'Breathing Retraining spent_minutes', 
    'Journal Writing spent_minutes', 
    'Adaptive Coping spent_minutes',
    'Common Irrational Thoughts spent_minutes', 
    'Choosing a Coping Strategy spent_minutes', 
    'Set a SMART Goal spent_minutes',
    'Meditation spent_minutes',
    ]

# compute mdn / iqr

summary = {}

for skill in skills_min:
    non_zero = d[d[skill] > 0][skill] ### parse for >0 values
    zero_count = (d[skill] == 0).sum() ### count 0 values
    mdn = non_zero.median()
    q1 = non_zero.quantile(0.25)
    q3 = non_zero.quantile(0.75)
    val_range = non_zero.max() - non_zero.min() ### range of >0 values
    
    summary[skill] = {
        'mdn': mdn,
#        'q1': q1,
#        'q3': q3,
        'zero_count': zero_count,
        'non_zero_range': val_range,       
        }

# tabulate, display

summary = pd.DataFrame(summary).T ### transpose for interpretability
print(summary)

In [ ]:
# * completion_count - mdn / iqr

skills_complete = [
    'Breathing Retraining completion_count', 
    'Journal Writing completion_count',
    'Adaptive Coping completion_count',
    'Common Irrational Thoughts completion_count',
    'Choosing a Coping Strategy completion_count',
    'Set a SMART Goal completion_count',
    'Meditation completion_count',
    ]

# compute m (sd)

summary = {}

for skill in skills_complete:
    non_zero = d[d[skill] > 0][skill] ### parse for >0 values
    zero_count = (d[skill] == 0).sum() ### count 0 values
#    mean_val = d[col].mean()
#    std_val = d[col].std()
    mean_val = non_zero.mean()
    std_val = non_zero.std()
    summary[skill] = {
        'mean': mean_val,
        'std': std_val,
        'zero_count': zero_count,
        }

# tabulate, display

summary = pd.DataFrame(summary).T
print(summary)

In [ ]:
# get n

sums = {}

for skill in skills_complete:
    total = d[skill].sum()
    sums[skill] = total

# tabulate, display

d_sums = pd.DataFrame.from_dict(
    sums, 
    orient = 'index', 
    columns = ['sum'],
    )
print(d_sums)


In [ ]:
# get %: n / N skills completed

19 / 205

### 3. Visualize
Creates scatter plots with jittered points and mean±SD overlays.
***

In [ ]:
import seaborn as sns

d = pd.read_csv(DATA_DIR / 'nola_gem_acceptability_no_loc_tx.csv', index_col = [0])

# inspect

d.shape
d.info()
d.head(3)

##### **Fig. 1.** NOLA Gem acceptability: overall impressions.

In [ ]:
# Fig. 1: overall impressions (int1-int5)

plot_acceptability_scatter(
    df = d,
    int_cols = ['int1_rev', 'int2_rev', 'int3_rev', 
              'int4_rev', 'int5_rev'],
    x_labels = ['easy to use', 'useful for coping', 'recommend to a friend', 
              'satisfied with app', 'use again'],
    y_labels = {1: 'disagree', 2: 'somewhat\ndisagree', 3: 'somewhat\nagree', 4: 'agree'},
    title = 'NOLA Gem acceptability: overall impressions',
    output_path = FIGURES_DIR / 'int1_int5_high_res_scatter.png',
    )

##### **Fig. 2a.** NOLA Gem acceptability: perceived helpfulness.

In [ ]:
# Fig. 2a: perceived helpfulness (int6-int10)

plot_acceptability_scatter(
    df = d,
    int_cols = ['int6_rev', 'int7_rev', 'int8_rev', 
              'int9_rev', 'int10_rev'],
    x_labels = ['reducing distress', 'improving mood', 'facilitating coping', 
              'changing habits', 'learning new skills'],
    y_labels = {1: 'disagree', 2: 'somewhat\ndisagree', 3: 'somewhat\nagree', 4: 'agree'},
    title = '($a.$) I found NOLA Gem to be helpful in...',
    output_path = FIGURES_DIR / 'int6_int10_high_res_scatter.png',
    )

##### **Fig. 2b.** NOLA Gem acceptability: perceived helpfulness of features.

In [ ]:
# Fig. 2b: perceived helpfulness of features (int11-int14)

plot_acceptability_scatter(
    df = d,
    int_cols = ['int11_rev', 'int12_rev', 'int13_rev', 
              'int14_rev'],
    x_labels = ['educational sessions', 'skills practice', 'geofencing alerts', 
              'daily diary suggested skills'],
    y_labels = {1: 'not very helpful', 2: 'a little\nhelpful', 3: 'somewhat\nhelpful', 4: 'very helpful'},
    title = '($b.$) I found the NOLA Gem __________ features to be...$^a$',
    output_path = FIGURES_DIR / 'int11_int14_high_res_scatter.png',
    replace_val = 5,  # handle N/A responses
)

### 4. Code
Local LLM x human synergistic deductive qualitative coding with on-device Ollama model (Gemma 12B).
***

#### 4a. Transform qual-ready df

In [ ]:
# create qual df from main data

d_qual = d[['open1', 'open2', 'open3', 'open4']].copy()

# slice by open-ended item

d_qual_open1 = d_qual[['open1']].copy()
d_qual_open2 = d_qual[['open2']].copy()
d_qual_open3 = d_qual[['open3']].copy()
d_qual_open4 = d_qual[['open4']].copy()

# inspect

d_qual.info()
d_qual.head(3)

In [ ]:
# insert qual code / rationale cols

d_qual[[
    'fctn_sjs', 'fctn_rtnl_sjs', 'lgth_sjs', 'lgth_rtnl_sjs',
    'tmng_sjs', 'tmng_rtnl_sjs', 'attn_sjs', 'attn_rtnl_sjs',
    'gltc_sjs', 'gltc_rtnl_sjs', 'prfc_sjs', 'prfc_rtnl_sjs',
    'ftrs_sjs', 'ftrs_rtnl_sjs', 'strs_sjs', 'strs_rtnl_sjs',
    ]] = ' '

d_qual.info()
d_qual.head(3)

In [ ]:
# specify 'item' col for long df

d_qual_open1['item'] = 'open1'
d_qual_open2['item'] = 'open2'
d_qual_open3['item'] = 'open3'
d_qual_open4['item'] = 'open4'

# rename all qual cols: 'text'

d_qual_open1 = d_qual_open1.rename(columns = {'open1': 'text'})
d_qual_open2 = d_qual_open2.rename(columns = {'open2': 'text'})
d_qual_open3 = d_qual_open3.rename(columns = {'open3': 'text'})
d_qual_open4 = d_qual_open4.rename(columns = {'open4': 'text'})

# concatenate: long df

d_qual_long = pd.concat(
    [d_qual_open1, d_qual_open2,
     d_qual_open3, d_qual_open4],
    axis = 0,
    ignore_index = False,
    )

# reorder

d_qual_long = d_qual_long.reindex(columns = ['item', 'text',
    'fctn_sjs', 'fctn_rtnl_sjs', 'lgth_sjs', 'lgth_rtnl_sjs', 'tmng_sjs', 'tmng_rtnl_sjs',
    'attn_sjs', 'attn_rtnl_sjs', 'gltc_sjs', 'gltc_rtnl_sjs', 'prfc_sjs', 'prfc_rtnl_sjs',
    'ftrs_sjs', 'ftrs_rtnl_sjs', 'strs_sjs', 'strs_rtnl_sjs',
    ])

d_qual_long.info()
d_qual_long.head(3)

In [ ]:
# export

d_qual_long.to_excel(DATA_DIR / 'nola_gem_acceptability_qual.xlsx')

#### 4b. Formulate deductive coding prompts

In [ ]:
# role / text template - shared across all prompts

role = '''
    You are tasked with applying pre-defined qualitative codes to segments of text excerpted 
    from interviews with users of a mental health app for people living with HIV. 
    
    The app included text-messaged surveys multiple times a day, as well as self-directed sessions 
    on mindfulness meditation, trauma psychoeducation (etc.), and momentary prompts to engage with 
    brief coping skills training as needed throughout the day. 
    
    You will be provided a definition, instructions, and 
    key exemplars of text to guide your coding decisions.
    '''

text = '''
    ---
    Text to classify:
    "{text}"
    ---
    '''

##### _EMA questionnaire length_ (alias: `lgth`)

In [ ]:
lgth_prompt = build_prompt(
    code = 'EMA questionnaire length',
    alias = 'lgth',
    code_def = '''
        Describes frustration with text message survey, daily diary, or questionnaire (all refer to the same 
        measure) length. These frustrations can include aspects such as the number of questions, or the amount 
        of time allotted to complete the questionnaire, or the nature of the questions administered to participants.
    
        Frustrations must be directly attributable to the duration of the survey or the time it takes to respond 
        to, specifically and explicitly. Vague dissatisfaction with the survey does not warrant a 'lgth' = 1. 
        ''',
    code_ex = '''
        - "The questions they texted you, there was just too many…the questions seemed to ask the same thing 
        over and again"
        - "The number of questions was too many. It's inconvenient especially if you have typical work and 
        family responsibilities"
        - "There are so many questions on the survey and they don't give you enough time to finish them all"
        ''',
    )

print(lgth_prompt)

##### _EMA prompt timing_ (alias: `tmng`)

In [ ]:
tmng_prompt = build_prompt(
    code = 'EMA prompt timing',
    alias = 'tmng',
    code_def = '''
        Describes frustration with text message survey, daily diary, or questionnaire (all refer to the same 
        measure) timing during the day. These frustrations can include the questionnaires being sent too early, 
        or too late, or being difficult to fit in due to obligations such as a full-time job.
    
        Frustrations must be directly attributable to the timing of the survey delivery, specifically
        and explicitly. Vague dissatisfaction with the survey does not warrant a 'tmng' = 1. 
        ''',
    code_ex = '''
         - "I couldn’t always answer the surveys when they sent them during the day…because I’ve got a 
         full-time job"
        - "The timing was all wrong. The diaries should come in earlier or later in the day so I can 
        answer without disrupting my schedule"
        - "The time periods for the text message surveys was unworkable for me. Timing would be better if 
        you could choose yourself"
        ''',
    )

print(tmng_prompt)

##### _EMA item attunement_ (alias: `attn`)

In [ ]:
attn_prompt = build_prompt(
    code = 'EMA item attunement',
    alias = 'attn',
    code_def = '''
        Recounts opinions that text message survey, daily diary, or questionnaire (all refer to the same 
        measure) were inappropriate to or incongruent with app users’ lived experience, including recommendations 
        for more appropriate questions or question wording.
    
        The text must refer to the mismatch between item wording and the user's life experience, explicitly and 
        specifically, to warrant a 'attn' = 1. Mentions of formatting issues such as the survey length or timing, 
        alone, do _not_ warrant a 'attn' = 1
        ''',
    code_ex = '''
        - "Not all of the questions made sense to me, or really fit into how I think about my life"
        - "The questions felt too vague. I wasn't sure what they meant"
        - "More questions on prayer and surivival, less about drugs and violence"
        ''',
    )

print(attn_prompt)

##### _"Gotta get those glitches fixed_ [in vivo]_"_ (alias: `gltc`)

In [ ]:
gltc_prompt = build_prompt(
    code = 'Glitch fixes',
    alias = 'gltc',
    code_def = '''
        Describes any app (including text message delivery) malfunctions.
        ''',
    code_ex = '''
        - "It just froze while I was answering my survey one day"
        - "I feel like it could be great but it felt too glitchy for now"
        - "The glitches. I just couldn't deal with the glitches"
        ''',
    )

print(gltc_prompt)

##### "_This app is perfect_ [in vivo]" (alias: `prfc`)

In [ ]:
prfc_prompt = build_prompt(
    code = 'Perfection',
    alias = 'prfc',
    code_def = '''
        Captures all responses that express satisfaction with the pilot app or reject any suggestions for 
        updated features.
    
        Positive sentiment towards the app, alone, does _not_ warrant a 'prfc' = 1; statements that nothing 
        needs to be changed must be specific and explicit to warrant a 'prfc' = 1.
        ''',
    code_ex = '''
        - "I don't think you need to put in any changes at all"
        - "There was nothing I didn't like"
        - "I liked it all"
        ''',
    )

print(prfc_prompt)

##### _Tweaks and fresh features_ (alias: `ftrs`)

In [ ]:
ftrs_prompt = build_prompt(
    code = 'Tweaks and fresh features',
    alias = 'ftrs',
    code_def = '''
        Captures all specific recommendations for alterations to existing app features or expressed wishes for 
        entirely new features to be added to the app.
    
        New features must be suggested clearly and specifically to warrant a 'ftrs' = 1; dissatisfaction with 
        existing features alone does _not_ warrant a 'ftrs' = 1.
        ''',
    code_ex = '''
        - "More coping skill should be added"
        - "I wish it had less text, more multimedia"
        - "The questions should be more personalized use to user to better fit our needs"
        ''',
    )

print(ftrs_prompt)

##### _“You seem a little stressed…it will tell you what you can do_ [in vivo]_”_ (alias: `strs`)

In [ ]:
strs_prompt = build_prompt(
    code = 'JITAI recognition',
    alias = 'strs',
    code_def = '''
        Captures any expression of appreciation for the app's features being offered when needed 
        throughout the day, or tailored to particular stressors reported by the user.
        
        Positive sentiment towards the app, alone, does _not_ warrant a 'strs' = 1; descriptions 
        of the app being uniquely responsive to users' needs must be specific and explicit.
        ''',
    code_ex = '''
        - "I liked how different meditations were offered for different stresses you mention throughout the day"
        - "The daily check-ins and quick follow ups to try and help you right when you most need it"
        - "Learning all these new skills to stay calm and collected during the day"
        ''',
    )

print(strs_prompt)

#### 4c. Dual code human-coded qual data

In [ ]:
d = pd.read_excel(DATA_QUAL / 'nola_gem_acceptability_qual_sjs.xlsx', index_col = [0])

In [ ]:
#%%capture

# locally hosted Ollama endpoint

ollama_endpoint = 'http://localhost:11434/api/generate'

# define aliases, prompts

prompts = [
    ('lgth', lgth_prompt), ### (alias, prompt_template)
    ('tmng', tmng_prompt),
    ('attn', attn_prompt),
    ('gltc', gltc_prompt),
    ('prfc', prfc_prompt),
    ('ftrs', ftrs_prompt),
    ('strs', strs_prompt),
    ]

# loop through each alias, prompt

for alias, prompt_template in prompts:

# apply code_texts_deductively_ollama over aliases, prompts, updated_df

    d_coded = code_texts_deductively_ollama(
        d,
        alias = alias,
        text_column = 'text',
        endpoint_url = ollama_endpoint,
        prompt_template = prompt_template,
        model_name = 'gemma3:12B',
        )

d_coded.head(3)

# export

d_coded.to_excel('d_coded.xlsx')

#### 4d. Compute Cohen's $\kappa$ / % agreement

In [ ]:
d = pd.read_excel(DATA_QUAL / 'd_coded.xlsx', index_col = [0])

encodings_sjs = [
    'lgth_sjs', 'tmng_sjs','attn_sjs','gltc_sjs',
    'prfc_sjs', 'ftrs_sjs', 'strs_sjs',
    ]

# numeric conversion - coerce

for e in encodings_sjs:
    d[e] = pd.to_numeric(d[e], errors = 'coerce')

# replace NaN w/ 0    
    
d[encodings_sjs] = d[encodings_sjs].fillna(0)

# inspect

d.info()
d.head(3)

In [ ]:
# define kappa fx

def calculate_kappa(d, col1, col2):
    return cohen_kappa_score(d[col1], d[col2])

col_pairs = [
    ('lgth_sjs', 'lgth_llm'), 
    ('tmng_sjs', 'tmng_llm'),
    ('attn_sjs', 'attn_llm'),
    ('gltc_sjs', 'gltc_llm'),
    ('prfc_sjs', 'prfc_llm'),
    ('ftrs_sjs', 'ftrs_llm'),
    ('strs_sjs', 'strs_llm'),
    ]

# initialize dict

kappa_results = {}

# % agreement loop

def calculate_percent_agreement(df, col_pairs):
    results = {}
    for col1, col2 in col_pairs:
        agreement = df[col1] == df[col2]
        percent_agreement = (agreement.sum() / len(df)) * 100
        results[f"{col1} & {col2}"] = percent_agreement
    return results

percent_agreement_results = calculate_percent_agreement(d, col_pairs)

for pair, percent in percent_agreement_results.items():
    print(f"Percent agreement for {pair}: {percent:.2f}%")

print("\n")

# kappa loop

for col1, col2 in col_pairs:
    kappa = calculate_kappa(d, col1, col2)
    kappa_results[f'{col1} and {col2}'] = kappa

for pair, kappa in kappa_results.items():
    print(f"Cohen's Kappa for {pair}: {kappa:.2f}")
    
print("\n")

In [ ]:
# compute Cohen's kappa for each code

codes = [
    'lgth', 'tmng', 'attn', 'gltc', 
    'prfc', 'ftrs', 'strs',
    ]

kappa_results = {}

for code in codes:
    sjs_col = f'{code}_sjs'
    llm_col = f'{code}_llm'
    
    if sjs_col in d.columns and llm_col in d.columns:
        # drop rows with NaN in either column
        valid_mask = d[sjs_col].notna() & d[llm_col].notna()
        sjs_ratings = d.loc[valid_mask, sjs_col].astype(int)
        llm_ratings = d.loc[valid_mask, llm_col].astype(int)
        
        if len(sjs_ratings) > 0:
            kappa = cohen_kappa_score(sjs_ratings, llm_ratings)
            kappa_results[code] = kappa
            print(f"{code}: kappa = {kappa:.3f} (n = {len(sjs_ratings)})")

# summary

d_kappa = pd.DataFrame.from_dict(
    kappa_results, 
    orient = 'index', 
    columns = ['kappa'],
    )

print("\nSummary:")
print(d_kappa)

In [ ]:
# flag disagreements

#print(d.columns.tolist())

def encode_disagreements(row):
    return 1 if row[0] != row[1] else 0

col_dis = [
    ('lgth_sjs', 'lgth_llm', 'lgth_dis'), 
    ('tmng_sjs', 'tmng_llm', 'tmng_dis'),
    ('attn_sjs', 'attn_llm', 'attn_dis'),
    ('gltc_sjs', 'gltc_llm', 'gltc_dis'),
    ('prfc_sjs', 'prfc_llm', 'prfc_dis'),
    ('strs_sjs', 'strs_llm', 'strs_dis'),
    ]

for col1, col2, dis_col in col_dis:
    d[dis_col] = d[[col1, col2]].apply(
        encode_disagreements,
        axis = 1,
        )

# obfuscate human vs. Gemma encodings    
    
# Example: rename _sjs to _a and _llm to _b

#d.columns = [
#    col.replace('_sjs', '_a').replace('_llm', '_b') if col.endswith(('_sjs', '_llm')) else col
#    for col in d.columns
#    ]

# reorder for ST interpretability 

d = d[[
    'item', 'text', 
    'lgth_sjs', 'lgth_rtnl_sjs', 'lgth_llm', 'lgth_expl','lgth_dis', 
    'tmng_sjs', 'tmng_rtnl_sjs', 'tmng_llm', 'tmng_expl', 'tmng_dis', 
    'attn_sjs', 'attn_rtnl_sjs', 'attn_llm', 'attn_expl', 'attn_dis', 
    'gltc_sjs', 'gltc_rtnl_sjs','gltc_llm', 'gltc_expl', 'gltc_dis', 
    'prfc_sjs', 'prfc_rtnl_sjs','prfc_llm', 'prfc_expl', 'prfc_dis', 
    'strs_sjs', 'strs_rtnl_sjs', 'strs_llm', 'strs_expl', 'strs_dis',
    ]]

# inspect

d.info()
d.head(3)    
        
# export

d.to_excel(f'd_coded_icr.xlsx')

> End of `nola_gem_acceptability.ipynb`